In [1]:
import pandas as pd
from surprise import Dataset
from surprise import Reader

from surprise import Reader, Dataset, SVD
from surprise.model_selection.validation import cross_validate
from sklearn.feature_extraction.text import TfidfVectorizer

from sklearn.metrics.pairwise import cosine_similarity
from scipy import sparse


import numpy as np

In [2]:
df_books = pd.read_csv('./ir_assignment_table_books.csv', encoding = "utf-8" )
df_books.head(10)

,ISBN,Book-Title,Book-Author,Year-Of-Publication,Publisher,Image-URL-S,Image-URL-M,Image-URL-L,id
0,0195153448,Classical Mythology,Mark P. O. Morford,2002,Oxford University Press,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...,1
1,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,2
2,0060973129,Decision in Normandy,Carlo D'Este,1991,HarperPerennial,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...,3
3,0374157065,Flu: The Story of the Great Influenza Pandemic...,Gina Bari Kolata,1999,Farrar Straus Giroux,http://images.amazon.com/images/P/0374157065.0...,http://images.amazon.com/images/P/0374157065.0...,http://images.amazon.com/images/P/0374157065.0...,4
4,0393045218,The Mummies of Urumchi,E. J. W. Barber,1999,W. W. Norton & Company,http://images.amazon.com/images/P/0393045218.0...,http://images.amazon.com/images/P/0393045218.0...,http://images.amazon.com/images/P/0393045218.0...,5
5,0399135782,The Kitchen God's Wife,Amy Tan,1991,Putnam Pub Group,http://images.amazon.com/images/P/0399135782.0...,http://images.amazon.com/images/P/0399135782.0...,http://images.amazon.com/images/P/0399135782.0...,6
6,0425176428,What If?: The World's Foremost Military Histor...,Robert Cowley,2000,Berkley Publishing Group,http://images.amazon.com/images/P/0425176428.0...,http://images.amazon.com/images/P/0425176428.0...,http://images.amazon.com/images/P/0425176428.0...,7
7,0671870432,PLEADING GUILTY,Scott Turow,1993,Audioworks,http://images.amazon.com/images/P/0671870432.0...,http://images.amazon.com/images/P/0671870432.0...,http://images.amazon.com/images/P/0671870432.0...,8
8,0679425608,Under the Black Flag: The Romance and the Real...,David Cordingly,1996,Random House,http://images.amazon.com/images/P/0679425608.0...,http://images.amazon.com/images/P/0679425608.0...,http://images.amazon.com/images/P/0679425608.0...,9
9,074322678X,Where You'll Find Me: And Other Stories,Ann Beattie,2002,Scribner,http://images.amazon.com/images/P/074322678X.0...,http://images.amazon.com/images/P/074322678X.0...,http://images.amazon.com/images/P/074322678X.0...,10


In [3]:
small_dataset = df_books['Book-Title'][0:50000]

In [4]:
vectorizer = TfidfVectorizer(dtype=np.float32)
X = vectorizer.fit_transform(small_dataset)


In [5]:
vectorizer.get_feature_names()

['10',
 '101',
 '10x18',
 '12',
 '13',
 '16',
 '18',
 '19',
 '1983',
 '1stbooks',
 '20',
 '2000',
 '21st',
 '22',
 '2nd',
 '300incredible',
 '3f',
 '3rd',
 '400',
 '61',
 '62',
 '84',
 'aah',
 'abacus',
 'abbeville',
 'abbey',
 'abc',
 'abdo',
 'abel',
 'aber',
 'abingdon',
 'able',
 'about',
 'abradale',
 'abrams',
 'absolute',
 'abson',
 'abstract',
 'ac',
 'acacia',
 'academic',
 'academy',
 'acantilado',
 'accelerated',
 'accent',
 'access',
 'accion',
 'acciã³n',
 'acclaim',
 'accord',
 'ace',
 'acmillan',
 'acolyte',
 'acropolis',
 'actar',
 'actes',
 'activa',
 'ad',
 'adagio',
 'adam',
 'adams',
 'addison',
 'adelphi',
 'admission',
 'adobe',
 'adult',
 'advance',
 'advancebooks',
 'advanced',
 'advancement',
 'advantage',
 'adventist',
 'adventure',
 'adventures',
 'aegina',
 'aerial',
 'aerie',
 'affairs',
 'africa',
 'ag',
 'aga',
 'agencja',
 'aglow',
 'aguilar',
 'ai',
 'aid',
 'aim',
 'aims',
 'air',
 'aird',
 'airmont',
 'aise',
 'ait',
 'ak',
 'akal',
 'akanos',
 'akash

In [6]:
X.shape

(50000, 4067)

In [7]:
X

<50000x4067 sparse matrix of type '<class 'numpy.float32'>'
	with 107405 stored elements in Compressed Sparse Row format>

In [8]:
matrix_size = X.data.nbytes + X.indptr.nbytes + X.indices.nbytes
matrix_size

1059244

In [9]:
print(X[3])

  (0, 1513)	0.5775226
  (0, 3523)	0.5770055
  (0, 1330)	0.5775226


In [10]:
len(vectorizer.get_feature_names())

4067

In [11]:
similarity_matrix = cosine_similarity(X, dense_output=False)
matrix_size = similarity_matrix.data.nbytes + similarity_matrix.indptr.nbytes + similarity_matrix.indices.nbytes
matrix_size

1718884596

In [12]:
similarity_matrix.count_nonzero()

214835574